In [1]:
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    BodyIndex,
    Box,
    DiagramBuilder,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    Parser,
    RigidTransform,
    Role,
    RollPitchYaw,
    Simulator,
    StartMeshcat,
)

# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at https://b934ab79-7769-49f9-b02a-120e5e511f36.deepnoteproject.com/7001/


## **Tuning MultibodyPlant for Simulation**

# Problem Description

In earlier homework problems, you were given the simulation environment to start with. Now, it is your turn to design a simulation! Simulations are incredibly important in robotic software development. The more accurate we can make our simulation, the better our controller can be and so on. In lecture, we learned about contact modeling and what the important parameters for a stable simulation are. In this exercise, you will investigate the underlying simulation details and get prepared for your project. Specifically, we will learn both how to reproduce desired simulation output and pinpoint simulation issues, in a classic physics example of two blocks stacking on a slope.

**These are the learning goals of this exercise:**
1. Learning to debug and engineer collision geometries.
2. Understand the relative importance of simulation parameters.

In [2]:
# Define size of blocks and the slope
slope = 0.1
q1 = [0.0, 0, 0.15]
q2 = [0.0, 0, 0.20]

# Block sizes
block_size1 = [0.02, 0.02, 0.02]  # Red block
block_size2 = [0.02, 0.02, 0.02]  # Green block

In [3]:
#######################################################
### Don't change/remove! For visualization purposes ###
#######################################################

nonstack_pose1 = [
    [9.94885056e-01, -1.55900179e-02, 9.98031851e-02, 1.95741356e-02],
    [1.56650268e-02, 9.99877295e-01, 3.21017596e-05, -1.15839339e-04],
    [-9.97914393e-02, 1.53148200e-03, 9.95007198e-01, 5.83171660e-02],
    [0.00000000e00, 0.00000000e00, 0.00000000e00, 1.00000000e00],
]

nonstack_pose2 = [
    [-1.00114398e-01, 5.89905683e-03, 9.94958446e-01, 2.11092651e-01],
    [4.44194866e-04, 9.99982590e-01, -5.88414908e-03, 2.24353720e-03],
    [-9.94975834e-01, -1.47132606e-04, -1.00115275e-01, 3.89206165e-02],
    [0.00000000e00, 0.00000000e00, 0.00000000e00, 1.00000000e00],
]


stack_pose1 = [
    [9.94996362e-01, -3.77459725e-05, 9.99111553e-02, 2.64605688e-01],
    [3.79942503e-05, 9.99999999e-01, -5.82200282e-07, -1.13024604e-06],
    [-9.99111552e-02, 4.37533660e-06, 9.94996362e-01, 3.33522993e-02],
    [0.00000000e00, 0.00000000e00, 0.00000000e00, 1.00000000e00],
]

stack_pose2 = [
    [9.94996470e-01, -1.41453220e-05, 9.99100858e-02, 2.66676737e-01],
    [1.41925281e-05, 1.00000000e00, 2.38281298e-07, -9.14267754e-07],
    [-9.99100858e-02, 1.18088765e-06, 9.94996470e-01, 5.22415534e-02],
    [0.00000000e00, 0.00000000e00, 0.00000000e00, 1.00000000e00],
]

#######################################################

#  Set up your MultibodyPlant
In practice, you may have a pre-packaged simulation engine that is doing its job. However, to make a simulation performant, you need to tune the right physics parameters, e.g. mass and frictions, for your specific application. Run the code below to set up the simulation testing code.

Note, we use [SDF (Simulation Description Format)](http://sdformat.org/spec) respresentations to construct objects and then load them directly into our simulation with MultibodyPlant. SDF files allow us to define an object or robot for simulation through an intuitive XML format. We can use them to specify custom geometry, friction, mass, and even create separate linkages / joints. Full documentation for this file format can be found [here](http://sdformat.org/spec). A similar XML file format exists called [URDF (Unified Robot Description Format)](http://wiki.ros.org/urdf/XML). You may find these resources useful for your final project.

In [4]:
ground_slope_SDF = f"""
<?xml version="1.0" ?>
<sdf version="1.6">
<model name="ground">
    <static>true</static>
    <link name="ground">
        <collision name="ground_collision">
            <pose>0 0 0 0 {slope} 0</pose>
            <geometry>
                <box>
                    <size>10 10 0.1</size>
                </box>
            </geometry>
            <surface>
                <friction>
                    <ode>
                        <mu>0.085</mu>
                        <mu2>0.085</mu2>
                    </ode>
                </friction>
            </surface>
        </collision>
        <visual name="ground_visual">
            <pose>0 0 0 0 {slope} 0</pose>
            <geometry>
                <box>
                    <size>10 10 0.1</size>
                </box>
            </geometry>
            <material>
                <diffuse>0.8 0.8 0.8 1.0</diffuse>
            </material>
        </visual>
    </link>
</model>
</sdf>
"""

In [5]:
def generate_block_sdf(shape, name, mass=1, mu=1, color=[0.5, 0.5, 0.9, 1.0]):
    return f"""
    <?xml version="1.0" ?>
    <sdf version="1.6">
    <model name="{name}">
        <static>false</static>
        <link name="{name}">
            <collision name="block_collision">
                <geometry>
                    <box>
                        <size>{shape.width()} {shape.depth()} {shape.height()}</size>
                    </box>
                </geometry>
                <surface>
                    <friction>
                        <ode>
                            <mu>{mu}</mu>
                            <mu2>{mu}</mu2>
                        </ode>
                    </friction>
                </surface>
            </collision>
            <visual name="block_visual">
                <geometry>
                    <box>
                        <size>{shape.width()} {shape.depth()} {shape.height()}</size>
                    </box>
                </geometry>
                <material>
                    <diffuse>{color[0]} {color[1]} {color[2]} {color[3]}</diffuse>
                </material>
            </visual>
            <inertial>
                <mass>{mass}</mass>
                <inertia>
                    <ixx>{1/12 * (shape.depth()**2 + shape.height()**2)}</ixx>
                    <iyy>{1/12 * (shape.width()**2 + shape.height()**2)}</iyy>
                    <izz>{1/12 * (shape.depth()**2 + shape.width()**2)}</izz>
                </inertia>
            </inertial>
        </link>
    </model>
    </sdf>
    """

In [6]:
def MakeSimulation(time_step, mass, mu, simulation_time=1.0):
    """
    Create simple simulation with blocks and a slope based on input parameters
    In this exercise, we will tune these parameters to achieve desired simulation results
    """
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=time_step)

    # Add slope to the MultibodyPlant using SDF
    parser = Parser(plant)
    parser.AddModelsFromString(ground_slope_SDF, ".sdf")

    # Add blocks to the MultibodyPlant using generated SDF
    block1_sdf = generate_block_sdf(
        Box(*block_size1),
        "block1",
        mass[0],
        mu[0],
        color=[0.8, 0, 0, 1.0],
    )
    parser.AddModelsFromString(block1_sdf, ".sdf")
    block2_sdf = generate_block_sdf(
        Box(*block_size2),
        "block2",
        mass[1],
        mu[1],
        color=[0, 0.8, 0, 1.0],
    )
    parser.AddModelsFromString(block2_sdf, ".sdf")

    nonstacking = simulation_time == 1
    if simulation_time == 1:
        pose1, pose2 = nonstack_pose1, nonstack_pose2
    else:
        pose1, pose2 = stack_pose1, stack_pose2

    # Visualize goal block positions for parameter tuning tasks
    plant.AddModelInstance("block1_goal")
    plant.RegisterVisualGeometry(
        plant.world_body(),
        RigidTransform(pose1),
        Box(*block_size1),
        "block1_goal",
        [0.8, 0, 0, 0.3],
    )
    plant.AddModelInstance("block2_goal")
    plant.RegisterVisualGeometry(
        plant.world_body(),
        RigidTransform(pose2),
        Box(*block_size2),
        "block2_goal",
        [0, 0.8, 0, 0.3],
    )

    # Build the plant and meshcat
    plant.Finalize()
    meshcat.Delete()
    meshcat.DeleteAddedControls()
    meshcat_param = MeshcatVisualizerParams()

    # kProximity for collision geometry and kIllustration for visual geometry
    meshcat_param.role = Role.kIllustration
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, scene_graph, meshcat, meshcat_param
    )

    meshcat.Set2dRenderMode(xmin=-0.2, xmax=0.2, ymin=-0.2, ymax=0.3)

    # Initialize simulation
    diagram = builder.Build()
    simulator = Simulator(diagram)
    if nonstacking:
        simulator.set_target_realtime_rate(0.5)  # slow motion!
    else:
        simulator.set_target_realtime_rate(3)  # fast motion!

    tf1 = RigidTransform(RollPitchYaw(0, 0, 0), q1)
    tf2 = RigidTransform(RollPitchYaw(0, 0, 0), q2)
    context = simulator.get_context()
    plant_context = diagram.GetSubsystemContext(plant, context)
    plant.SetFreeBodyPose(plant_context, plant.get_body(BodyIndex(2)), tf1)
    plant.SetFreeBodyPose(plant_context, plant.get_body(BodyIndex(3)), tf2)

    # Simulate and visualize
    visualizer.StartRecording()
    simulator.AdvanceTo(simulation_time)
    visualizer.StopRecording()
    visualizer.PublishRecording()

    return simulator, diagram

(2pt) a. Run the code block below. We notice that the green block is falling past the red block and the thin slope. Why? Anwer in your written submission.

> Because the timestep is large then we can simulate the contact dynamics in an accurate manner.

(2pt) b. There are specific algorithms that can help avoid pass-through events in simulation, which we haven't implemented yet here. But even without them, we can address the issue by tuning the timesteps. Try tuning the simulation timestep in `set_sim_timestep` such that the both block falls on (but not inside) the slope. Should we decrease or increase the timestep if we want more accurate contact dynamics? Note that if the simulation advances too fast, you can pause and reset simulation to check the initial state.

> We should decrease the timestep for more accurate contact dynamics because of the high stiffness.

HINT: You should only change timestep value by factors of 10.

(NOTE: If the simulation advances too fast, you can pause and reset simulation to check the initial state.)

In [23]:
def set_sim_timestep():
    """
    Tune simulation timestep such that the blocks do not fall under the slope.
    """
    time_step = 0.001  # MODIFY HERE
    mass1, mass2 = 0.01, 5
    return time_step, (mass1, mass2), (0.1, 0.1)


simulator_b, diagram_b = MakeSimulation(*set_sim_timestep())

# Sliding blocks

(2pt) c. Changing the mass of simulated objects will of course impact how the simulation behaves. In our simple simulation, we can imagine changing mass will most notably impact the magnitude of gravitational force on the objects. Let's try to reproduce the following gif by tuning the masses of the blocks in `set_block_masses`.

HINT: You should only change mass values by factors of 10.

<img src="https://raw.githubusercontent.com/RussTedrake/manipulation/master/book/figures/exercises/sim_tuning_final_state1.gif" width="700"> 

In [36]:
q1 = [0.0, 0, 0.065]  # Initial pose of the red block
q2 = [0.0, 0, 0.087]  # Initial pose of the green block


def set_block_masses():
    """
    Tune block masses to match the gif above.
    """
    mass1, mass2 = 0.01, 0.01  # MODIFY HERE
    mu1, mu2 = 0.1, 0.1
    return 0.001, (mass1, mass2), (mu1, mu2)


simulator_c, diagram_c = MakeSimulation(*set_block_masses())

# Stacking blocks
(3pt) d. We define the slope angle as $\alpha$ and the mass of each block to be $m_1$ (top block) and $m_2$ (bottom block). The friction between the blocks is defined by $\mu_1$, and the friction between the slope surface and the bottom block is $\mu_2$. 

Assume the two cube blocks are on top of each other on the slope. What constraint do we need on the friction parameters $\mu_1,\mu_2$ to satisfy such that the top block is sticking to the bottom block (no relative velocity) and the bottom block is sliding down the slope at a constant speed? Which coefficient must be greater and do they depend on the mass $m_1,m_2$ of the blocks? Draw out the free-body [diagram](https://youtu.be/N19SU7vgX7c?t=3394) to derive an answer. Write your answer in written submission.

> $\mu_1$ has to resist $m_1$. $\mu_2$ has to resist total mass. $\mu_2 > \mu_1$

(3pt) e. Changing the coefficient of friction between simulated object will impact how they behave in contact will each other. Try to reproduce the simulated result below by changing the friction values in `set_friction_coeffs`. 

HINT: You should only change values by factors of 10.

<img src="https://raw.githubusercontent.com/RussTedrake/manipulation/master/book/figures/exercises/sim_tuning_final_state2.gif" width="700"> 

In [39]:
q1 = [0.0, 0, 0.065]  # Initial pose of the red block
q2 = [0.0, 0, 0.087]  # Initial pose of the green block


def set_friction_coeffs():
    """
    Choose friction coefficients to make the blocks stack and move as shown above.
    """
    time_step = 0.00001
    mass1, mass2 = 0.1, 0.05
    mu1, mu2 = 0.1, 1  # MODIFY HERE
    return time_step, (mass1, mass2), (mu1, mu2)


# We have to make simulator timestep really small, still the blocks slide relative to each other a little bit.
# Why? Check the last section here https://drake.mit.edu/doxygen_cxx/group__contact__engineering.html
simulator_e, diagram_e = MakeSimulation(*set_friction_coeffs(), simulation_time=1.9)

## How will this notebook be Graded?

If you are enrolled in the class, this notebook will be graded using [Gradescope](www.gradescope.com). You should have gotten the enrollement code on our announcement in Piazza. 

For submission of this assignment, you must do two things. 
- Download and submit the notebook `simulation_tuning.ipynb` to Gradescope's notebook submission section, along with your notebook for the other problems.
- Write down your answers to 5.6 to a separately pdf file and submit it to Gradescope's written submission section. 

We will evaluate the local functions in the notebook to see if the function behaves as we have expected. For this exercise, the rubric is as follows:
- [2 pts] 5.6.a  Reason for block falling is answered correctly and attached to written submission. 
- [2 pts] 5.6.b `set_sim_timestep` is implemented correctly. 
- [2 pts] 5.6.c `set_block_masses` is implemented correctly.
- [3 pts] 5.6.d  The analysis is answered correctly and attached to written submission.
- [3 pts] 5.6.e `set_friction_coeffs` is implemented correctly.

In [40]:
from manipulation.exercises.clutter.test_simulation_tuning import TestSimulationTuning
from manipulation.exercises.grader import Grader

Grader.grade_output([TestSimulationTuning], [locals()], "results.json")
Grader.print_test_results("results.json")

Total score is 7/7.

Score for Test test_set_friction is 3/3.

Score for Test test_set_masses is 2/2.

Score for Test test_set_timestep is 2/2.


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b934ab79-7769-49f9-b02a-120e5e511f36' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>